## TF

In [1]:
collection = [
    ('Document1', 'This is a sample'),
    ('Document2', 'This is another sample')
]

# in-memory (Hash Key 값)
# 전체 색인어 목록(Dictionary)
# {단어1:포스팅위치, 단어2:포스팅위치, ...}
queryPosting = dict()

# 전체 문서 목록(Dictionary)
# [0:문서1, 1:문서2, ...]
globalDocument = list()

# disk
# 사전에 있는 색인어 중, 어느 문서에서, 몇 번 나타났는지
# [(단어 idx, 문서 idx, 빈도, 다음주소), ...]
# [0:Tuple(lexiconIdx, documentIdx, freq, 다음포스팅위치-fptr)]
# 메모리 X, File OK
globalPosting = list()

In [2]:
for (docName, docContent) in collection:
    # Pointer 대체용, Key, Document 이름은 절대로 겹치지 않는다는 가정
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    # {단어idx:빈도, 단어idx:빈도, ...}
    localPosting = dict()
    
    # Local 작업
    for term in docContent.lower().split():
        # Local에 대해서, 없으면 추가
        if term not in localPosting.keys():
            localPosting[term] = 1
        # 있으면, 빈도 증가
        else:
            localPosting[term] += 1
     
    # Global Marge
    # fp -> struct(단어, 빈도) (localPosting)
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in queryPosting.keys(): 
            lexiconIdx = len(queryPosting)
            postingIdx = len(globalPosting) # fseek
            postingData = (lexiconIdx, docIdx, termFreq, -1)
            globalPosting.append(postingData)
            queryPosting[indexTerm] = postingIdx # globalPosting 위치(ptr:idx)
        else: # 기존 단어의 idx 가져오기
            lexiconIdx = list(queryPosting.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = queryPosting[indexTerm]
            postingData = (lexiconIdx, docIdx, termFreq, beforeIdx)
            globalPosting.append(postingData)
            queryPosting[indexTerm] = postingIdx
            
#     print(localPosting)
# print(globalDocument)

#         if term not in globalLexicon.keys():
#             lexiconIdx = len(globalLexicon) 0

In [3]:
queryPosting, globalDocument

({'this': 4, 'is': 5, 'a': 2, 'sample': 7, 'another': 6},
 ['Document1', 'Document2'])

In [4]:
globalPosting

[(0, 0, 1, -1),
 (1, 0, 1, -1),
 (2, 0, 1, -1),
 (3, 0, 1, -1),
 (0, 1, 1, 0),
 (1, 1, 1, 1),
 (4, 1, 1, -1),
 (3, 1, 1, 3)]

In [5]:
for indexTerm, postingIdx in queryPosting.items():
    # indexTerm:단어: postingIdx:위치, ...
    print(indexTerm)
    
    while True: # Posting Next:-1
        if postingIdx == -1:
            break
            
        postingData = globalPosting[postingIdx]
        print('  DocName:{0} - TermFreq:{1} - Next:{2}'.format(globalDocument[postingData[1]], postingData[2], postingData[3]))
        postingIdx = postingData[3]
        
    print()

this
  DocName:Document2 - TermFreq:1 - Next:0
  DocName:Document1 - TermFreq:1 - Next:-1

is
  DocName:Document2 - TermFreq:1 - Next:1
  DocName:Document1 - TermFreq:1 - Next:-1

a
  DocName:Document1 - TermFreq:1 - Next:-1

sample
  DocName:Document2 - TermFreq:1 - Next:3
  DocName:Document1 - TermFreq:1 - Next:-1

another
  DocName:Document2 - TermFreq:1 - Next:-1



In [6]:
globalPosting[queryPosting['sample']]

(3, 1, 1, 3)

In [7]:
globalPosting[globalPosting[queryPosting['sample']][3]]   # 다음 주소가 "-1" 일때 까지 반복해서 찾음

(3, 0, 1, -1)

## TF-IDF

In [14]:
collection = [
    ('Document1', 'This is a sample'),
    ('Document2', 'This is another sample'),
    ('Document3', 'This is not sample'),
    ('Document4', 'a not'),
    ('Document5', 'not'),
    ('Document6', 'not sample')
]

In [9]:
from math import log10

def binaryTF(freq):
    if freq > 0:
        return 1
    else:
        return 0

def rawTF(freq):
    return freq

def basicTF(freq, totalFreq):
    return freq/totalFreq

def logTF(freq):
    if freq > 0:
        return 1+log10(freq)
    else:
        return 0

def doubleNormalTF(K, freq, maxFreq): 
    return K + ((1-K) * (freq/maxFreq))

## IDF(Inverse Document Frequency) 구하기

In [25]:
def unaryIDF():
    return 1

def basicIDF(N, df):
    return log10(N/df)

def smoothigIDF(N, df):
    return log10((N+1)/df)

def probabilityIDF(N, df):
    return log10((N-df+1)/df)

In [20]:
globalLexicon = dict()
globalDocument = list()
globalPosting = list()

for (docName, docContent) in collection:
    docIdx = len(globalDocument) # Pointer 대체용, Key, Document 이름은 절대로 겹치지 않는다는 가정
    globalDocument.append(docName)
    localPosting = dict() # {단어idx:빈도, 단어idx:빈도, ...}
    
    for term in docContent.lower().split():
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    maxFreq = max(localPosting.values())
    
    # Global Marge
    # fp -> struct(단어, 빈도) (localPosting)
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting) # fseek
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx # globalPosting 위치(ptr:idx)
        else: # 기존 단어의 idx 가져오기 
            # tuple로 리턴받을경우 immutable(읽기전용)이라 list로 바꿔 주어야 함.
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), beforeIdx]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx

In [21]:
globalPosting

[[0, 0, 1.0, -1],
 [1, 0, 1.0, -1],
 [2, 0, 1.0, -1],
 [3, 0, 1.0, -1],
 [0, 1, 1.0, 0],
 [1, 1, 1.0, 1],
 [4, 1, 1.0, -1],
 [3, 1, 1.0, 3],
 [0, 2, 1.0, 4],
 [1, 2, 1.0, 5],
 [5, 2, 1.0, -1],
 [3, 2, 1.0, 7],
 [2, 3, 1.0, 2],
 [5, 3, 1.0, 10],
 [5, 4, 1.0, 13],
 [5, 5, 1.0, 14],
 [3, 5, 1.0, 11]]

In [18]:
N = len(globalDocument)
globalLexiconIDF = dict()

for indexTerm, postingIdx in globalLexicon.items():
    df = 0
    oldPostingIdx = postingIdx
    
    while True:  
        if postingIdx == -1:
            break
        
        df += 1
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        
    idf = basicIDF(N, df)
    globalLexiconIDF[indexTerm] = idf
    postingIdx = oldPostingIdx
    
    print('{0} / IDF-{1}'.format(indexTerm, idf))
    
    while True:
        if postingIdx == -1:
            break
        
        postingData = globalPosting[postingIdx]
        TF = postingData[2]
        globalPosting[postingIdx][2] *= idf
        print('    Documents:{0} / TF:{1} / TF-IDF:{2}'.format(globalDocument[postingData[1]], 
                                                               TF, globalPosting[postingIdx][2]))
        
        postingIdx = postingData[3]

this / IDF-0.3010299956639812
    Documents:Document3 / TF:1.0 / TF-IDF:0.3010299956639812
    Documents:Document2 / TF:1.0 / TF-IDF:0.3010299956639812
    Documents:Document1 / TF:1.0 / TF-IDF:0.3010299956639812
is / IDF-0.3010299956639812
    Documents:Document3 / TF:1.0 / TF-IDF:0.3010299956639812
    Documents:Document2 / TF:1.0 / TF-IDF:0.3010299956639812
    Documents:Document1 / TF:1.0 / TF-IDF:0.3010299956639812
a / IDF-0.47712125471966244
    Documents:Document4 / TF:1.0 / TF-IDF:0.47712125471966244
    Documents:Document1 / TF:1.0 / TF-IDF:0.47712125471966244
sample / IDF-0.17609125905568124
    Documents:Document6 / TF:1.0 / TF-IDF:0.17609125905568124
    Documents:Document3 / TF:1.0 / TF-IDF:0.17609125905568124
    Documents:Document2 / TF:1.0 / TF-IDF:0.17609125905568124
    Documents:Document1 / TF:1.0 / TF-IDF:0.17609125905568124
another / IDF-0.7781512503836436
    Documents:Document2 / TF:1.0 / TF-IDF:0.7781512503836436
not / IDF-0.17609125905568124
    Documents:Docum

In [22]:
query = 'this is a sample'# 쿼리도 문서인 것처럼 생각한다. 

queryPosting = dict()

for term in query.lower().split(): # 소문자로 변환한 후(lower) split으로 쪼개기
    if term not in queryPosting.keys():
        queryPosting[term] = 1
    else:
        queryPosting[term] += 1
        
maxFreq = max(queryPosting.values())

for indexTerm, termFreq in queryPosting.items():
    queryPosting[indexTerm] = doubleNormalTF(0.5, termFreq, maxFreq) 

In [23]:
queryPosting

{'this': 1.0, 'is': 1.0, 'a': 1.0, 'sample': 1.0}

## euclidian Distance(맨하튼거리)로 검색하기

In [47]:
def euclidian(x, y):
    return (x-y) **2

In [48]:
candidateList = dict()

for indexTerm, postingIdx in globalLexicon.items():
    queryTFIDF = 0 # 일종의 weight역할
    
    # keys에 있는지 검사 후 query가 있을 경우에 한해 처리하기 위함
    if indexTerm in queryPosting.keys():
        queryTFIDF = queryPosting[indexTerm] * globalLexiconIDF[indexTerm]
    
    while True: # ptr Next: -1 
        if postingIdx == -1:
            break
        
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        documentWeight = postingData[2]
        
        # 키 에러 방지용
        if postingData[1] not in candidateList.keys():
            candidateList[postingData[1]] = euclidian(queryTFIDF, documentWeight)
        else:
            candidateList[postingData[1]] += euclidian(queryTFIDF, documentWeight)          


In [49]:
candidateList

{2: 0.3889488152499714,
 1: 1.0440720193766682,
 0: 0.32988132214072974,
 3: 0.05906749310924163,
 5: 0.11813498621848326,
 4: 0.05906749310924163}

In [67]:
# 정렬 (reversed = False는 작은값부터 큰 값 순서)
resultList = sorted(candidateList.items(), key = lambda x:x[1])

print(collection) # 검색하고자 하는 문서 이름과 내용
print("Current Query : ", [query])

for i, (documentIdx, distance) in enumerate(resultList):
    print('순위:{0}. 문서:{1} / 거리:{2}'.format((i+1), globalDocument[documentIdx],distance)) # 해당하는 문서(documentIdx)도 출력
    print('  {0}'.format(collection[documentIdx][1])) 
    
# query가 not일때와 a일때를 비교해 보면 유클리디안 거리의 문제점이 드러난다.(원노트 참고) 따라서 cosine simmilarity를 주로 사용한다. 

[('Document1', 'This is a sample'), ('Document2', 'This is another sample'), ('Document3', 'This is not sample'), ('Document4', 'a not'), ('Document5', 'not'), ('Document6', 'not sample')]
Current Query :  ['a']
순위:1. 문서:Document4 / 거리:0.05906749310924163
  a not
순위:2. 문서:Document5 / 거리:0.05906749310924163
  not
순위:3. 문서:Document6 / 거리:0.11813498621848326
  not sample
순위:4. 문서:Document1 / 거리:0.32988132214072974
  This is a sample
순위:5. 문서:Document3 / 거리:0.3889488152499714
  This is not sample
순위:6. 문서:Document2 / 거리:1.0440720193766682
  This is another sample


### Smoothing

In [51]:
globalLexicon = dict()
globalDocument = list()
globalPosting = list()

for (docName, docContent) in collection:
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    localPosting = dict()
    
    for term in docContent.lower().split():
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    maxFreq = max(localPosting.values())
    
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting)
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), beforeIdx]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx

In [52]:
N = len(globalDocument)
globalLexiconIDF = dict()
globalDocumentLength = dict()

for indexTerm, postingIdx in globalLexicon.items():
    df = 0
    oldPostingIdx = postingIdx
    
    while True:  
        if postingIdx == -1:
            break
        
        df += 1
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        
    idf = smoothigIDF(N, df)
    globalLexiconIDF[indexTerm] = idf
    postingIdx = oldPostingIdx
    
    print('{0} / IDF-{1}'.format(indexTerm, idf))
    
    while True:
        if postingIdx == -1:
            break
        
        postingData = globalPosting[postingIdx]
        TF = postingData[2]
        globalPosting[postingIdx][2] *= idf
        print('    Documents:{0} / TF:{1} / TF-IDF:{2}'.format(globalDocument[postingData[1]], 
                                                               TF, globalPosting[postingIdx][2]))
        
        postingIdx = postingData[3]
        
        if postingData[1] not in globalDocumentLength.keys():
            globalDocumentLength[postingData[1]] = globalPosting[postingIdx][2] ** 2
        else:
            globalDocumentLength[postingData[1]] += globalPosting[postingIdx][2] ** 2

this / IDF-0.36797678529459443
    Documents:Document3 / TF:1.0 / TF-IDF:0.36797678529459443
    Documents:Document2 / TF:1.0 / TF-IDF:0.36797678529459443
    Documents:Document1 / TF:1.0 / TF-IDF:0.36797678529459443
is / IDF-0.36797678529459443
    Documents:Document3 / TF:1.0 / TF-IDF:0.36797678529459443
    Documents:Document2 / TF:1.0 / TF-IDF:0.36797678529459443
    Documents:Document1 / TF:1.0 / TF-IDF:0.36797678529459443
a / IDF-0.5440680443502757
    Documents:Document4 / TF:1.0 / TF-IDF:0.5440680443502757
    Documents:Document1 / TF:1.0 / TF-IDF:0.5440680443502757
sample / IDF-0.24303804868629444
    Documents:Document6 / TF:1.0 / TF-IDF:0.24303804868629444
    Documents:Document3 / TF:1.0 / TF-IDF:0.24303804868629444
    Documents:Document2 / TF:1.0 / TF-IDF:0.24303804868629444
    Documents:Document1 / TF:1.0 / TF-IDF:0.24303804868629444
another / IDF-0.8450980400142568
    Documents:Document2 / TF:1.0 / TF-IDF:0.8450980400142568
not / IDF-0.24303804868629444
    Documents:

In [58]:
query = 'a' # Document처럼 생각

queryPosting = dict()

for term in query.lower().split():
    if term not in queryPosting.keys():
        queryPosting[term] = 1
    else:
        queryPosting[term] += 1
        # queryPosting => {단어:빈도, 단어:빈도, ...}
        
maxFreq = max(queryPosting.values())

for indexTerm, termFreq in queryPosting.items():
    queryPosting[indexTerm] = doubleNormalTF(0.5, termFreq, maxFreq) 

In [65]:
queryPosting

{'a': 1.0}

In [59]:
candidateList = dict()

for indexTerm, postingIdx in globalLexicon.items(): # keys에 있는지 검사 후 query가 있을 경우에 한해 처리하기 위함
    queryTFIDF = 0
    
    if indexTerm in queryPosting.keys():
        queryTFIDF = queryPosting[indexTerm] * globalLexiconIDF[indexTerm]
    
    while True: # ptr Next: -1
        if postingIdx == -1:
            break
        
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        documentWeight = postingData[2]
        
        # 키 에러 방지용
        if postingData[1] not in candidateList.keys():
            candidateList[postingData[1]] = euclidian(queryTFIDF, documentWeight)
        else:
            candidateList[postingData[1]] += euclidian(queryTFIDF, documentWeight)

In [60]:
candidateList

{2: 0.3889488152499714,
 1: 1.0440720193766682,
 0: 0.32988132214072974,
 3: 0.05906749310924163,
 5: 0.11813498621848326,
 4: 0.05906749310924163}

In [61]:
resultList = sorted(candidateList.items(), key = lambda x:x[1])
print('query: ', query)

for i, (documentIdx, distance) in enumerate(resultList):
    print('순위:{0}. 문서:{1} / 거리:{2}'.format((i+1), globalDocument[documentIdx],distance))
    print('  {0}'.format(collection[documentIdx][1]))

query:  a
순위:1. 문서:Document4 / 거리:0.05906749310924163
  a not
순위:2. 문서:Document5 / 거리:0.05906749310924163
  not
순위:3. 문서:Document6 / 거리:0.11813498621848326
  not sample
순위:4. 문서:Document1 / 거리:0.32988132214072974
  This is a sample
순위:5. 문서:Document3 / 거리:0.3889488152499714
  This is not sample
순위:6. 문서:Document2 / 거리:1.0440720193766682
  This is another sample


## Cosine Simmilarity로 검색 수행하기

In [62]:
def innerProduct(x, y): # 내적 연산
    return x * y

In [63]:
globalLexicon = dict()
globalDocument = list()
globalPosting = list()

for (docName, docContent) in collection:
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    localPosting = dict()
    
    for term in docContent.lower().split():
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    maxFreq = max(localPosting.values())
    
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting)
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), beforeIdx]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx

In [64]:
N = len(globalDocument)
globalLexiconIDF = dict()
globalDocumentLength = dict()

for indexTerm, postingIdx in globalLexicon.items():
    # indexTerm : 단어, postingIdx :위치(포인터역할), ...
    df = 0
    oldPostingIdx = postingIdx
    
    while True: # ptr Next: -1 
        if postingIdx == -1:
            break
        
        df += 1
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        
    idf = smoothigIDF(N, df)
    globalLexiconIDF[indexTerm] = idf # 나중에 단어가 나왔을때 idf를 불러올 수 있도록 하기 위함
    postingIdx = oldPostingIdx
    print('단어 : {0} / IDF : {1}'.format(indexTerm, idf))
    
    while True:
        if postingIdx == -1:
            break
        
        postingData = globalPosting[postingIdx]
        TF = postingData[2]
        globalPosting[postingIdx][2] *= idf
        print('    Documents:{0} / TF:{1} / TF-IDF:{2}'.format(globalDocument[postingData[1]], 
                                                               TF, globalPosting[postingIdx][2]))
        
        postingIdx = postingData[3]
        
        # 색인하는 과정에서 사실상 검색까지 모두 한 셈이 되므로 색인 과정에만 시간이 들어 굉장히 빠른 연산속도를 보임. 
        # 키 에러 방지용(키값이 있다면 제곱해서 누적해주면 됨)
        if postingData[1] not in globalDocumentLength.keys():
            globalDocumentLength[postingData[1]] = globalPosting[postingIdx][2] ** 2
        else:
            globalDocumentLength[postingData[1]] += globalPosting[postingIdx][2] ** 2

this / IDF-0.36797678529459443
    Documents:Document3 / TF:1.0 / TF-IDF:0.36797678529459443
    Documents:Document2 / TF:1.0 / TF-IDF:0.36797678529459443
    Documents:Document1 / TF:1.0 / TF-IDF:0.36797678529459443
is / IDF-0.36797678529459443
    Documents:Document3 / TF:1.0 / TF-IDF:0.36797678529459443
    Documents:Document2 / TF:1.0 / TF-IDF:0.36797678529459443
    Documents:Document1 / TF:1.0 / TF-IDF:0.36797678529459443
a / IDF-0.5440680443502757
    Documents:Document4 / TF:1.0 / TF-IDF:0.5440680443502757
    Documents:Document1 / TF:1.0 / TF-IDF:0.5440680443502757
sample / IDF-0.24303804868629444
    Documents:Document6 / TF:1.0 / TF-IDF:0.24303804868629444
    Documents:Document3 / TF:1.0 / TF-IDF:0.24303804868629444
    Documents:Document2 / TF:1.0 / TF-IDF:0.24303804868629444
    Documents:Document1 / TF:1.0 / TF-IDF:0.24303804868629444
another / IDF-0.8450980400142568
    Documents:Document2 / TF:1.0 / TF-IDF:0.8450980400142568
not / IDF-0.24303804868629444
    Documents:

In [66]:
query = 'a' # Document

queryPosting = dict()

for term in query.lower().split():
    if term not in queryPosting.keys():
        queryPosting[term] = 1
    else:
        queryPosting[term] += 1
        
maxFreq = max(queryPosting.values())

for indexTerm, termFreq in queryPosting.items():
    queryPosting[indexTerm] = doubleNormalTF(0.5, termFreq, maxFreq)

In [68]:
candidateList = dict()

for indexTerm, queryWeight in queryPosting.items(): # keys에 있는지 검사 후 query가 있을 경우에 한해 처리하기 위함
    if indexTerm in globalLexicon.keys():
        postingIdx = globalLexicon[indexTerm]
        
        while True: # ptr Next: -1
            if postingIdx == -1:
                break
            
            postingData = globalPosting[postingIdx]
            postingIdx = postingData[3]
            documentWeight = postingData[2]
            
            
            # 키 에러 방지용
            if postingData[1] not in candidateList.keys():
                candidateList[postingData[1]] = innerProduct(queryWeight, documentWeight)
            else:
                candidateList[postingData[1]] += innerProduct(queryWeight, documentWeight)

for documentIdx, sumProduct in candidateList.items():
    candidateList[documentIdx] /= globalDocumentLength[documentIdx]

In [69]:
# 정렬 (reversed = False는 작은값부터 큰 값 순서)
resultList = sorted(candidateList.items(), key=lambda x:x[1], reverse=True)

print('query: ', query)

for i, (documentIdx, distance) in enumerate(resultList):
    print('순위:{0} / 문서:{1} / 유사도:{2}'.format((i+1), globalDocument[documentIdx], distance))
    print('   document:{0}'.format(collection[documentIdx][1]))

query:  a
순위:1 / 문서:Document4 / 유사도:0.27203402217513784
   document:a not
순위:2 / 문서:Document1 / 유사도:0.1778542139314762
   document:This is a sample


## TDM-DTM-TWM

In [70]:
from konlpy.corpus import kobill
from collections import defaultdict
from konlpy.tag import Kkma

ma = Kkma().morphs

In [71]:
def getDocReprByDefaultDict():
    docRepr = defaultdict(lambda: defaultdict(int))
    
    for docName in kobill.fileids():
        document = kobill.open(docName).read()
 
        for token in document.split(): 
            for morpheme in ma(token):
                docRepr[docName][morpheme] += 1
    
    return docRepr

In [73]:
DTM = getDocReprByDefaultDict()

In [74]:
DTM

defaultdict(<function __main__.getDocReprByDefaultDict.<locals>.<lambda>()>,
            {'1809890.txt': defaultdict(int,
                         {'지방': 9,
                          '공무원': 16,
                          '법': 7,
                          '일부': 4,
                          '개정': 8,
                          '법률안': 3,
                          '(': 27,
                          '정의': 3,
                          '화': 2,
                          '의원': 2,
                          '대표': 1,
                          '발의': 2,
                          ')': 27,
                          '의': 55,
                          '안': 4,
                          '벌': 1,
                          'ㄴ': 23,
                          '호': 8,
                          '9890': 1,
                          '연월일': 1,
                          ':': 10,
                          '2010': 3,
                          '.': 53,
                          '11': 2,
                          '12': 1,


In [87]:
def convertInvertedDocument(DTM):
    TDM = defaultdict(lambda: defaultdict(int))
    
    for fileName, termList in DTM.items():  
        maxFreq = max(termList.values())
        for term, freq in termList.items():
            TDM[term][fileName] = doubleNormalTF(0, freq, maxFreq)
            
    return TDM # 빈도와 가중치를 동시에 갖게 된다.

In [78]:
TDM = convertInvertedDocument(DTM) # 0322_main.ipynb와 달리 빈도와 가중치를 동시에 갖게 된다.
TDM

defaultdict(<function __main__.convertInvertedDocument.<locals>.<lambda>()>,
            {'지방': defaultdict(int,
                         {'1809890.txt': 0.07317073170731707,
                          '1809891.txt': 0.04201680672268908,
                          '1809892.txt': 0.025,
                          '1809893.txt': 0.05434782608695652,
                          '1809897.txt': 0.015625,
                          '1809898.txt': 0.014285714285714285}),
             '공무원': defaultdict(int,
                         {'1809890.txt': 0.13008130081300814,
                          '1809891.txt': 0.13445378151260504,
                          '1809892.txt': 0.095,
                          '1809893.txt': 0.09782608695652174,
                          '1809896.txt': 0.01293103448275862}),
             '법': defaultdict(int,
                         {'1809890.txt': 0.056910569105691054,
                          '1809891.txt': 0.058823529411764705,
                          '1809892.txt': 

In [79]:
# Term - Document => Term-Weight
N = len(DTM)

def TDM2TWM(TDM): # TDM을 받아 term에 대한 weight가 들어있는 값으로 리턴
    TWM = defaultdict(lambda: defaultdict(float))
    DVL = defaultdict(float)
    
    for term, tfList in TDM.items():
        df = len(tfList)
        idf = basicIDF(N, df)
        
        for fileName, tf in tfList.items():
            # 각 문서에서 각 단어가 차지하는 단어
            TWM[term][fileName] = tf * idf
            DVL[fileName] += TWM[term][fileName] ** 2
        
    return TWM, DVL

In [80]:
TWM, DVL = TDM2TWM(TDM)

In [81]:
TWM # 0322_main.ipynb와 달리 빈도와 가중치를 동시에 갖게 된다.

defaultdict(<function __main__.TDM2TWM.<locals>.<lambda>()>,
            {'지방': defaultdict(float,
                         {'1809890.txt': 0.016232835337782175,
                          '1809891.txt': 0.00932137603430069,
                          '1809892.txt': 0.00554621874040891,
                          '1809893.txt': 0.0120569972617585,
                          '1809897.txt': 0.0034663867127555687,
                          '1809898.txt': 0.003169267851662234}),
             '공무원': defaultdict(float,
                         {'1809890.txt': 0.03915837341970488,
                          '1809891.txt': 0.04047462126574537,
                          '1809892.txt': 0.028597849588078213,
                          '1809893.txt': 0.02944858653234599,
                          '1809896.txt': 0.0038926292542756187}),
             '법': defaultdict(float,
                         {'1809890.txt': 0.005515203992328414,
                          '1809891.txt': 0.005700589000473907,
       

In [82]:
DVL

defaultdict(float,
            {'1809890.txt': 0.08891068576179642,
             '1809891.txt': 0.09407896411798794,
             '1809892.txt': 0.056003860773669055,
             '1809893.txt': 0.1488797397661453,
             '1809897.txt': 0.8414240759234841,
             '1809898.txt': 0.5833111894640503,
             '1809896.txt': 0.2372859866006518,
             '1809894.txt': 0.5467267027461252,
             '1809895.txt': 0.35539863400824295,
             '1809899.txt': 0.08503881498928936})

In [83]:
query = '국방의 의무와 보편적 교육에 대한 법안'

queryRepr = defaultdict(int)

for token in query.split():
    for morpheme in ma(token):
        queryRepr[morpheme] += 1 # 빈도도 쉽게 구할 수 있다. 
        
maxFreq = max(queryRepr.values())
queryWeight = defaultdict(float)

for token, freq in queryRepr.items():
    if token in TWM.keys():
        tf = doubleNormalTF(0.5, freq, maxFreq)
        df = len(TWM[token])
        idf = basicIDF(N, df)
        queryWeight[token] = tf * idf

In [84]:
queryWeight

defaultdict(float,
            {'국방': 0.6989700043360189,
             '의': 0.0,
             '의무': 0.6989700043360189,
             '와': 0.04575749056067514,
             '교육': 0.09691001300805642,
             '에': 0.0,
             '대하': 0.0,
             'ㄴ': 0.0,
             '법안': 0.3979400086720376})

In [85]:
from math import sqrt

candidateList = defaultdict(float)

for token, weight in queryWeight.items():
    for fileName, tfidf in TWM[token].items():
        print('{0} : {1} = {2} * {3}'.format(token, fileName, weight, tfidf))
        candidateList[fileName] += innerProduct(weight, tfidf)
    
for fileName, sumProduct in candidateList.items():
    candidateList[fileName] /= sqrt(DVL[fileName])

국방 : 1809897.txt = 0.6989700043360189 * 0.02184281263550059
국방 : 1809898.txt = 0.6989700043360189 * 0.00998528577622884
의 : 1809890.txt = 0.0 * 0.0
의 : 1809891.txt = 0.0 * 0.0
의 : 1809892.txt = 0.0 * 0.0
의 : 1809893.txt = 0.0 * 0.0
의 : 1809894.txt = 0.0 * 0.0
의 : 1809895.txt = 0.0 * 0.0
의 : 1809896.txt = 0.0 * 0.0
의 : 1809897.txt = 0.0 * 0.0
의 : 1809898.txt = 0.0 * 0.0
의 : 1809899.txt = 0.0 * 0.0
의무 : 1809896.txt = 0.6989700043360189 * 0.0060256034856553346
의무 : 1809899.txt = 0.6989700043360189 * 0.0016255116379907415
와 : 1809890.txt = 0.04575749056067514 * 0.0011160363551384182
와 : 1809891.txt = 0.04575749056067514 * 0.0011535501822018943
와 : 1809892.txt = 0.04575749056067514 * 0.0006863623584101271
와 : 1809893.txt = 0.04575749056067514 * 0.0014920920835002763
와 : 1809895.txt = 0.04575749056067514 * 0.0031923830623726843
와 : 1809896.txt = 0.04575749056067514 * 0.001972305627615308
와 : 1809897.txt = 0.04575749056067514 * 0.0028598431600421964
와 : 1809898.txt = 0.04575749056067514 * 0.0

In [86]:
from nltk.tokenize import sent_tokenize

resultList = sorted(candidateList.items(), key=lambda x:x[1], reverse=True)
K = 5

for i, (fileName, similarity) in enumerate(resultList):
    if i < K:
        print('Rank:{0} / Name:{1} / Similarity:{2:.4f}'.format((i+1), fileName, similarity))
        content = kobill.open(fileName).read()
        print(sent_tokenize(content)[:3])
        print()

Rank:1 / Name:1809897.txt / Similarity:0.0189
['국군부대의 아랍에미리트(UAE)군 교육훈련 지원 등에 \n관한 파견 동의안\n\n의안\n                                                                  제출연월일 :  2010.', '11.', '15.']

Rank:2 / Name:1809898.txt / Similarity:0.0092
['국군부대의 소말리아 해역 파견연장 동의안\n\n의안\n                                                                  제출연월일 :  2010.', '11.', '15.']

Rank:3 / Name:1809896.txt / Similarity:0.0090
['행정절차법 일부개정법률안\n\n(유선호의원 대표발의 )\n\n 의 안\n 번 호\n\n9896\n\n발의연월일 : 2010.', '11.', '15.']

Rank:4 / Name:1809892.txt / Similarity:0.0063
['교육공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9892\n\n발의연월일 : 2010.', '11.', '12.']

Rank:5 / Name:1809893.txt / Similarity:0.0057
['남녀고용평등과 일 ·가정 양립 지원에 관한 법률 \n\n일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9893\n\n발의연월일 : 2010.', '11.', '12.']

